In [1]:
from google.colab import files
uploaded = files.upload()

Saving olist_customers_dataset.csv to olist_customers_dataset.csv
Saving olist_geolocation_dataset.csv to olist_geolocation_dataset.csv
Saving olist_order_items_dataset.csv to olist_order_items_dataset.csv
Saving olist_order_payments_dataset.csv to olist_order_payments_dataset.csv
Saving olist_order_reviews_dataset.csv to olist_order_reviews_dataset.csv
Saving olist_orders_dataset.csv to olist_orders_dataset.csv
Saving olist_products_dataset.csv to olist_products_dataset.csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset.csv
Saving product_category_name_translation.csv to product_category_name_translation.csv


In [2]:
import pandas as pd
import numpy as np
from google.colab import files
from sqlalchemy import create_engine
import os

In [3]:
print("Uploaded files:", list(uploaded.keys()))

Uploaded files: ['olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_items_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_orders_dataset.csv', 'olist_products_dataset.csv', 'olist_sellers_dataset.csv', 'product_category_name_translation.csv']


In [4]:
required_files = [
    'olist_customers_dataset.csv',
    'olist_sellers_dataset.csv',
    'olist_order_reviews_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_products_dataset.csv',
    'olist_geolocation_dataset.csv',
    'product_category_name_translation.csv',
    'olist_orders_dataset.csv',
    'olist_order_payments_dataset.csv'
]

# Check if all files are present
missing_files = [f for f in required_files if f not in uploaded]
if missing_files:
    print(f"Missing files: {missing_files}")
    print("Please re-upload all required files")
else:
    print("All required files uploaded successfully")

# Load datasets with explicit file paths
df_olist_customers = pd.read_csv('olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('olist_sellers_dataset.csv')
df_olist_order_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_olist_order_items = pd.read_csv('olist_order_items_dataset.csv')
df_olist_products = pd.read_csv('olist_products_dataset.csv')
df_olist_geolocation = pd.read_csv('olist_geolocation_dataset.csv')
df_product_category_name_translation = pd.read_csv('product_category_name_translation.csv')
df_olist_orders = pd.read_csv('olist_orders_dataset.csv')
df_olist_order_payments = pd.read_csv('olist_order_payments_dataset.csv')

print("DataFrames loaded successfully!")

All required files uploaded successfully
DataFrames loaded successfully!


In [5]:
# Step 3: Create SQLite database
engine = create_engine('sqlite:///brazilian_ecommerce.db', echo=False)

# Step 4: Export DataFrames to SQL tables
datasets = {
    'olist_customers': df_olist_customers,
    'olist_sellers': df_olist_sellers,
    'olist_order_reviews': df_olist_order_reviews,
    'olist_order_items': df_olist_order_items,
    'olist_products': df_olist_products,
    'olist_geolocation': df_olist_geolocation,
    'product_translation': df_product_category_name_translation,
    'olist_orders': df_olist_orders,
    'olist_order_payments': df_olist_order_payments
}

for name, df in datasets.items():
    df.to_sql(name, con=engine, index=False, if_exists='replace')
    print(f"Exported {name} ({len(df)} rows) to database")

# Step 5: Verify database setup
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", engine)
print("\nTables in database:")
print(tables)

# Test query
test_query = "SELECT * FROM olist_customers LIMIT 3"
test_result = pd.read_sql(test_query, engine)
print("\nTest query result:")
print(test_result)

Exported olist_customers (99441 rows) to database
Exported olist_sellers (3095 rows) to database
Exported olist_order_reviews (99224 rows) to database
Exported olist_order_items (112650 rows) to database
Exported olist_products (32951 rows) to database
Exported olist_geolocation (1000163 rows) to database
Exported product_translation (71 rows) to database
Exported olist_orders (99441 rows) to database
Exported olist_order_payments (103886 rows) to database

Tables in database:
                   name
0       olist_customers
1         olist_sellers
2   olist_order_reviews
3     olist_order_items
4        olist_products
5     olist_geolocation
6   product_translation
7          olist_orders
8  olist_order_payments

Test query result:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  06

In [6]:
# Query 1: Count and Percentage of Jan 2018 Orders with 5-Star Reviews
query1 = """
SELECT
  COUNT(*) AS total_orders,
  SUM(CASE WHEN rev.review_score = 5 THEN 1 ELSE 0 END) AS five_star_orders,
  ROUND(100.0 * SUM(CASE WHEN rev.review_score = 5 THEN 1 ELSE 0 END) / COUNT(*), 2) AS percentage
FROM olist_orders ord
JOIN olist_order_reviews rev ON ord.order_id = rev.order_id
WHERE STRFTIME('%Y-%m', ord.order_purchase_timestamp) = '2018-01'
"""
result1 = pd.read_sql(query1, engine)
print("Query 1 Results: Jan 2018 Orders with 5-Star Reviews")
print(result1)
print("\n" + "="*80 + "\n")

Query 1 Results: Jan 2018 Orders with 5-Star Reviews
   total_orders  five_star_orders  percentage
0          7245              4097       56.55




In [7]:
# Query 2: YoY Purchase Trends
query2 = """
SELECT
  STRFTIME('%Y', order_purchase_timestamp) AS year,
  COUNT(DISTINCT customer_id) AS unique_customers,
  COUNT(*) AS total_orders,
  ROUND(SUM(pay.payment_value), 2) AS total_revenue
FROM olist_orders ord
JOIN olist_order_payments pay ON ord.order_id = pay.order_id
GROUP BY year
ORDER BY year
"""
result2 = pd.read_sql(query2, engine)
print("Query 2 Results: Year-over-Year Purchase Trends")
print(result2)
print("\n" + "="*80 + "\n")

Query 2 Results: Year-over-Year Purchase Trends
   year  unique_customers  total_orders  total_revenue
0  2016               328           346       59362.34
1  2017             45101         47525     7249746.73
2  2018             54011         56015     8699763.05




In [8]:
# Query 3: Average Order Value per Customer
query3 = """
SELECT
  customer_id,
  ROUND(AVG(payment_value), 2) AS avg_order_value
FROM (
  SELECT ord.customer_id, pay.payment_value
  FROM olist_orders ord
  JOIN olist_order_payments pay ON ord.order_id = pay.order_id
)
GROUP BY customer_id
ORDER BY avg_order_value DESC
LIMIT 10  -- Showing top 10 for brevity
"""
result3 = pd.read_sql(query3, engine)
print("Query 3 Results: Top 10 Customers by Average Order Value")
print(result3)
print("\n" + "="*80 + "\n")

Query 3 Results: Top 10 Customers by Average Order Value
                        customer_id  avg_order_value
0  1617b1357756262bfa56ab541c47bc16         13664.08
1  ec5b2ba62e574342386871631fafd3fc          7274.88
2  c6e2731c5b391845f6800c97401a43a9          6929.31
3  f48d464a0baaea338cb25f816991ab1f          6922.21
4  3fd6777bbce08a352fddd04e4a7cc8f6          6726.66
5  05455dfa7cd02f13d132aa7a6a9729c6          6081.54
6  df55c14d1476a9a3467f131269c2477f          4950.34
7  e0a2412720e9ea4f26c1ac985f6a7358          4809.44
8  24bbf5fd2f2e1b359ee7de94defc4a15          4764.34
9  3d979689f636322c62418b6346b1c6d2          4681.78




In [9]:
# Query 4: Top 5 Revenue Cities (2016-2018)
query4 = """
SELECT
  c.customer_city AS city,
  ROUND(SUM(pay.payment_value), 2) AS total_revenue
FROM olist_orders ord
JOIN olist_customers c ON ord.customer_id = c.customer_id
JOIN olist_order_payments pay ON ord.order_id = pay.order_id
WHERE STRFTIME('%Y', order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY city
ORDER BY total_revenue DESC
LIMIT 5
"""
result4 = pd.read_sql(query4, engine)
print("Query 4 Results: Top 5 Cities by Revenue (2016-2018)")
print(result4)
print("\n" + "="*80 + "\n")

Query 4 Results: Top 5 Cities by Revenue (2016-2018)
             city  total_revenue
0       sao paulo     2203373.09
1  rio de janeiro     1161927.36
2  belo horizonte      421765.12
3        brasilia      354216.78
4        curitiba      247392.48




In [10]:
# Query 5: State Revenue Table (2016-2018)
query5 = """
SELECT
  c.customer_state AS state,
  STRFTIME('%Y', order_purchase_timestamp) AS year,
  ROUND(SUM(pay.payment_value), 2) AS revenue
FROM olist_orders ord
JOIN olist_customers c ON ord.customer_id = c.customer_id
JOIN olist_order_payments pay ON ord.order_id = pay.order_id
WHERE STRFTIME('%Y', order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY state, year
ORDER BY year, revenue DESC
"""
result5 = pd.read_sql(query5, engine)
print("Query 5 Results: State Revenue by Year (2016-2018)")
print(result5)
print("\n" + "="*80 + "\n")

Query 5 Results: State Revenue by Year (2016-2018)
   state  year   revenue
0     SP  2016  16885.54
1     RJ  2016  13407.58
2     MG  2016   5642.97
3     RS  2016   4790.70
4     SC  2016   2730.42
..   ...   ...       ...
70    RO  2018  29382.48
71    AM  2018  14935.32
72    AP  2018   8931.27
73    RR  2018   7852.79
74    AC  2018   6683.10

[75 rows x 3 columns]




In [11]:
# Query 6: Top Sellers
query6 = """
SELECT
  s.seller_id,
  COUNT(i.order_item_id) AS items_sold,
  ROUND(SUM(i.price + i.freight_value), 2) AS revenue,
  COUNT(DISTINCT ord.customer_id) AS customers,
  AVG(rev.review_score) AS avg_rating
FROM olist_order_items i
JOIN olist_sellers s ON i.seller_id = s.seller_id
JOIN olist_orders ord ON i.order_id = ord.order_id
JOIN olist_order_reviews rev ON ord.order_id = rev.order_id
GROUP BY s.seller_id
ORDER BY revenue DESC
LIMIT 10  -- Showing top 10
"""
result6 = pd.read_sql(query6, engine)
print("Query 6 Results: Top 10 Sellers")
print(result6)
print("\n" + "="*80 + "\n")

Query 6 Results: Top 10 Sellers
                          seller_id  items_sold    revenue  customers  \
0  4869f7a5dfa277a7dca6462dcf3b52b2        1148  248102.61       1124   
1  7c67e1448b00f6e969d365cea6b010ab        1367  239610.34        976   
2  4a3ca9315b744ce9f8e9374361493884        1984  235581.68       1785   
3  53243585a1d6dc2643021fd1853d8905         408  233703.75        356   
4  fa1c13f2614d7b5c4749cbc52fecda94         582  202744.74        581   
5  da8622b14eb17ae2831f4ac5b9dab84a        1568  187200.57       1308   
6  7e93a43ef30c4f03f38b393420bc753a         339  182487.13        335   
7  1025f0e2d44d7041d6cf58b6550e0bfa        1431  173453.79        907   
8  7a67c85e85bb2ce8582c35f2203ad736        1166  161927.66       1151   
9  955fee9216a65b617aa5c0531780ce60        1489  159194.52       1277   

   avg_rating  
0    4.122822  
1    3.348208  
2    3.803931  
3    4.075980  
4    4.340206  
5    4.071429  
6    4.206490  
7    3.849755  
8    4.234991  
9   

In [12]:
# Query 7: Delivery Success Rate by State
query7 = """
SELECT
  c.customer_state AS state,
  COUNT(*) AS total_orders,
  SUM(CASE WHEN order_status = 'delivered' THEN 1 ELSE 0 END) AS successful_deliveries,
  ROUND(100.0 * SUM(CASE WHEN order_status = 'delivered' THEN 1 ELSE 0 END) / COUNT(*), 2) AS success_rate
FROM olist_orders ord
JOIN olist_customers c ON ord.customer_id = c.customer_id
GROUP BY state
ORDER BY success_rate DESC
"""
result7 = pd.read_sql(query7, engine)
print("Query 7 Results: Delivery Success Rate by State")
print(result7)
print("\n" + "="*80 + "\n")

Query 7 Results: Delivery Success Rate by State
   state  total_orders  successful_deliveries  success_rate
0     AC            81                     80         98.77
1     AP            68                     67         98.53
2     ES          2033                   1995         98.13
3     MS           715                    701         98.04
4     AM           148                    145         97.97
5     TO           280                    274         97.86
6     RS          5466                   5345         97.79
7     RN           485                    474         97.73
8     MT           907                    886         97.68
9     PR          5045                   4923         97.58
10    MG         11635                  11354         97.58
11    SC          3637                   3546         97.50
12    DF          2140                   2080         97.20
13    PA           975                    946         97.03
14    SP         41746                  40501       

In [13]:
# Query 8: Preferred Payment Methods by Category
query8 = """
SELECT
  t.product_category_name_english AS category,
  pay.payment_type,
  COUNT(*) AS payment_count,
  ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (PARTITION BY t.product_category_name_english), 2) AS percentage
FROM olist_order_payments pay
JOIN olist_order_items i ON pay.order_id = i.order_id
JOIN olist_products p ON i.product_id = p.product_id
JOIN product_translation t ON p.product_category_name = t.product_category_name
GROUP BY category, payment_type
ORDER BY category, payment_count DESC
"""
result8 = pd.read_sql(query8, engine)
print("Query 8 Results: Preferred Payment Methods by Category")
print(result8)
print("\n" + "="*80 + "\n")

Query 8 Results: Preferred Payment Methods by Category
                       category payment_type  payment_count  percentage
0    agro_industry_and_commerce  credit_card            145       57.54
1    agro_industry_and_commerce       boleto             60       23.81
2    agro_industry_and_commerce      voucher             42       16.67
3    agro_industry_and_commerce   debit_card              5        1.98
4              air_conditioning  credit_card            222       73.51
..                          ...          ...            ...         ...
258                        toys   debit_card             45        1.05
259               watches_gifts  credit_card           4858       78.34
260               watches_gifts       boleto           1025       16.53
261               watches_gifts      voucher            255        4.11
262               watches_gifts   debit_card             63        1.02

[263 rows x 4 columns]




In [19]:
# Query 9: Distance Between Cities - Corrected Version
from sqlalchemy import text

# First create temporary table with average geolocations
with engine.connect() as connection:
    connection.execute(text("""
        CREATE TEMP TABLE IF NOT EXISTS avg_geolocation AS
        SELECT
          geolocation_zip_code_prefix,
          AVG(geolocation_lat) AS lat,
          AVG(geolocation_lng) AS lng
        FROM olist_geolocation
        GROUP BY geolocation_zip_code_prefix
    """))
    connection.commit()

# Then calculate distances
query9 = """
SELECT
  c.customer_city,
  s.seller_city,
  ROUND(6371 * ACOS(
    SIN(RADIANS(c_geo.lat)) * SIN(RADIANS(s_geo.lat)) +
    COS(RADIANS(c_geo.lat)) * COS(RADIANS(s_geo.lat)) *
    COS(RADIANS(s_geo.lng) - RADIANS(c_geo.lng))
  ), 2) AS distance_km,
  COUNT(*) AS order_count
FROM olist_orders ord
JOIN olist_customers c ON ord.customer_id = c.customer_id
JOIN olist_order_items i ON ord.order_id = i.order_id
JOIN olist_sellers s ON i.seller_id = s.seller_id
JOIN avg_geolocation c_geo ON c.customer_zip_code_prefix = c_geo.geolocation_zip_code_prefix
JOIN avg_geolocation s_geo ON s.seller_zip_code_prefix = s_geo.geolocation_zip_code_prefix
GROUP BY c.customer_city, s.seller_city
ORDER BY distance_km DESC
LIMIT 10  -- Longest distances
"""

result9 = pd.read_sql(query9, engine)
print("Query 9 Results: Top 10 Longest Delivery Distances Between Cities")
print(result9)
print("\n" + "="*80 + "\n")

Query 9 Results: Top 10 Longest Delivery Distances Between Cities
            customer_city           seller_city  distance_km  order_count
0      ilha dos valadares      sao bento do sul      8677.91            1
1         porto trombetas        ribeirao preto      5338.62            2
2                ibiajara             sao paulo      4791.61            1
3  santo antonio do canaa       mogi das cruzes      3927.41            1
4               boa vista             bombinhas      3579.89            1
5        vila dos cabanos          praia grande      3543.88            1
6        vila dos cabanos           santo andre      3509.07            1
7        vila dos cabanos                 salto      3470.99            1
8               boa vista    fazenda rio grande      3399.19            1
9               boa vista  sao jose dos pinhais      3387.36            1


